In [ ]:
# Databricks notebook source
# MAGIC %md This notebook is slightly modified version of `MLflow Training Tutorial` from [MLflow examples](https://github.com/mlflow/mlflow/tree/master/examples/sklearn_elasticnet_wine).
# MAGIC
# MAGIC It predicts the quality of wine using [sklearn.linear_model.ElasticNet](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html).
# MAGIC This is a base code and will be modified further during the `Databricks: Reproducible experiments with MLflow and Delta Lake` tutorial.
# MAGIC
# MAGIC Attribution
# MAGIC * The data set used in this example is from http://archive.ics.uci.edu/ml/datasets/Wine+Quality
# MAGIC * P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis.
# MAGIC * Modeling wine preferences by data mining from physicochemical properties. In Decision Support Systems, Elsevier, 47(4):547-553, 2009.

In [24]:
import os
import warnings
import sys

import pandas as pd
import numpy as np
from abc import ABC, abstractmethod
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn

from sklearn.ensemble import RandomForestRegressor
import logging

In [25]:
class TrainParameter(ABC):
    
    def __init__(self):
        pass

    @abstractmethod
    def log_param():
        pass

    @abstractmethod
    def get_model():
        pass

class LrTrainingParameter(TrainParameter):

    name = "random forest regressor"
    def __init__(self, alpha = 0.5, l1_ratio = 0.5):
        super().__init__()
        self.alpha = alpha
        self.l1_ratio = l1_ratio

    def log_param(self):
        mlflow.log_param("alpha", self.alpha)
        mlflow.log_param("l1_ratio", self.l1_ratio)
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (self.alpha, self.l1_ratio))

    def get_model(self):
        return ElasticNet(alpha = self.alpha, l1_ratio = self.l1_ratio, random_state=42)

class RfTrainingParameter(TrainParameter):

    name = "linear regression"
    def __init__(self, max_depth = 2, n_estimators=100):
        super().__init__()
        self.max_depth = max_depth
        self.n_estimators = n_estimators

    def log_param(self):
        mlflow.log_param("max_depth", self.max_depth)
        mlflow.log_param("n_estimators", self.n_estimators)

    def get_model(self):
        return RandomForestRegressor(max_depth = self.max_depth, n_estimators = self.n_estimators, random_state=42)


In [26]:
# Wine Quality Sample
def train(parameters : TrainParameter):

    if not isinstance(parameters, TrainParameter):
        raise TypeError('Model choice was not typed correctly for the available development')

    def eval_metrics(actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    logger = logging.getLogger(__name__)
    # Read the wine-quality csv file from the URL
    csv_url =\
        'http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
    try:
        data = pd.read_csv(csv_url, sep=';')
    except Exception as e:
        logger.exception(
            "Unable to download training & test CSV, check your internet connection. Error: %s", e)

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = np.ravel(train[["quality"]])
    test_y = test[["quality"]]


    with mlflow.start_run():
        # Execute ElasticNet
        model = parameters.get_model()
        model.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = model.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)
        
        parameters.log_param()

        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("mae", mae)
        mlflow.log_metric("r2", r2)

        mlflow.sklearn.log_model(model, parameters.name)
        
        
        # Print out metrics
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

In [27]:
# Start the training
train(LrTrainingParameter(alpha=0.1, l1_ratio=0.9))
train(LrTrainingParameter(alpha=0.1, l1_ratio=0.7))
train(RfTrainingParameter(max_depth = 10, n_estimators = 50))
train(RfTrainingParameter(max_depth = 10, n_estimators = 30))

Elasticnet model (alpha=0.100000, l1_ratio=0.900000):
  RMSE: 0.7697086906662156
  MAE: 0.5961942437025146
  R2: 0.1706716101667891
Elasticnet model (alpha=0.100000, l1_ratio=0.700000):
  RMSE: 0.7083694204635825
  MAE: 0.5519483749026798
  R2: 0.2037572003087752
  RMSE: 0.5747370614965268
  MAE: 0.43689810156448644
  R2: 0.44784598274923504
  RMSE: 0.5652434250013335
  MAE: 0.4254573811025375
  R2: 0.4799590974449838
